In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from difflib import SequenceMatcher, HtmlDiff
import json
from IPython.display import display, HTML
import os
import re
import sqlite3
from datetime import datetime
from dotenv import load_dotenv

# Load environment variables
load_dotenv()


# LLM Report Comparison Tool
#
# This notebook helps you compare financial reports generated by different LLM providers
# (OpenAI vs Google) to verify differences and analyze the outputs.

# Database connection function
def get_db_connection():
    """Connect to the SQLite database specified in the environment variables"""
    from app.database import DATABASE_URL
    db_path = DATABASE_URL.replace("sqlite:///", "")
    return sqlite3.connect(db_path)


def get_memos_by_provider():
    """Fetch memos from the database grouped by LLM provider"""
    conn = get_db_connection()

    # Query to get all financial memos with their LLM provider
    query = """
            SELECT id,
                   company_name,
                   report_date,
                   summary,
                   analysis,
                   recommendations,
                   file_path,
                   llm_provider,
                   created_at
            FROM financial_memos
            ORDER BY created_at DESC \
            """

    df = pd.read_sql_query(query, conn)
    conn.close()

    # Group by provider
    return {
        'openai': df[df['llm_provider'] == 'openai'],
        'google': df[df['llm_provider'] == 'google'],
        'unknown': df[df['llm_provider'].isnull()]
    }


def compare_reports_for_same_document(document_id=None):
    """
    Compare reports generated by different LLMs for the same input document

    If document_id is None, it will find documents that have reports from multiple providers
    """
    conn = get_db_connection()

    if document_id is None:
        # Find documents that have reports from multiple providers
        query = """
                SELECT company_data_id, COUNT(DISTINCT llm_provider) as provider_count
                FROM financial_memos
                WHERE llm_provider IS NOT NULL
                GROUP BY company_data_id
                HAVING provider_count > 1 \
                """
        multi_provider_docs = pd.read_sql_query(query, conn)

        if len(multi_provider_docs) == 0:
            conn.close()
            return "No documents found with reports from multiple LLM providers"

        # Use the first document that has multiple providers
        document_id = multi_provider_docs.iloc[0]['company_data_id']

    # Get reports for this document
    query = f"""
    SELECT id, company_name, summary, analysis, recommendations, llm_provider
    FROM financial_memos
    WHERE company_data_id = {document_id}
    """

    reports = pd.read_sql_query(query, conn)
    conn.close()

    if len(reports) < 2:
        return f"Only found {len(reports)} report(s) for document ID {document_id}"

    return reports


def detect_llm_patterns(text):
    """
    Analyze text to detect patterns that might indicate which LLM generated it
    Returns a dictionary of pattern counts and confidence score
    """
    patterns = {
        'openai': [
            r'\bI think\b',  # GPT models often use "I think"
            r'\bAs an AI\b',  # GPT self-references
            r'\bHowever,\b',  # Tendency to use contrastive language
            r'\bIt\'s important to note\b',  # Common GPT phrase
        ],
        'google': [
            r'\bBased on the information\b',  # Common in Google models
            r'\bThe document indicates\b',  # More formal, document-focused language
            r'\bAccording to\b',  # More attribution-focused
            r'\bIt appears that\b',  # More tentative language
        ]
    }

    results = {}
    for provider, pattern_list in patterns.items():
        matches = 0
        for pattern in pattern_list:
            matches += len(re.findall(pattern, text, re.IGNORECASE))
        results[provider] = matches

    # Calculate confidence score
    total_matches = sum(results.values())
    confidence = {}

    if total_matches > 0:
        for provider, matches in results.items():
            confidence[provider] = matches / total_matches

    return {
        'pattern_matches': results,
        'confidence': confidence
    }


def display_report_comparison(reports_df):
    """Display a side-by-side comparison of reports from different providers"""
    if isinstance(reports_df, str):
        return HTML(f"<div>{reports_df}</div>")

    providers = reports_df['llm_provider'].unique()

    if len(providers) < 2:
        return HTML(f"<div>Need at least 2 providers to compare. Found: {', '.join(providers)}</div>")

    # Create a comparison for each section
    sections = ['summary', 'analysis', 'recommendations']

    html_output = f"<h2>Report Comparison for {reports_df.iloc[0]['company_name']}</h2>"

    for section in sections:
        html_output += f"<h3>{section.title()}</h3>"
        html_output += "<table style='width:100%; border-collapse: collapse;'>"

        # Header row
        html_output += "<tr>"
        for provider in providers:
            html_output += f"<th style='border:1px solid black; padding:8px; background-color:#f2f2f2;'>{provider}</th>"
        html_output += "</tr>"

        # Content row
        html_output += "<tr>"
        for provider in providers:
            provider_report = reports_df[reports_df['llm_provider'] == provider].iloc[0]
            content = provider_report[section].replace('\n', '<br>')

            # Run pattern detection
            patterns = detect_llm_patterns(provider_report[section])
            confidence = patterns['confidence']
            confidence_html = ""

            if provider in confidence and confidence[provider] > 0:
                conf_pct = confidence[provider] * 100
                color = "green" if conf_pct > 70 else "orange"
                confidence_html = f"<div style='color:{color};'><small>Confidence: {conf_pct:.1f}%</small></div>"

            html_output += f"<td style='border:1px solid black; padding:8px; vertical-align:top;'>{confidence_html}{content}</td>"
        html_output += "</tr>"

        html_output += "</table>"

        # Add a diff view
        if len(providers) == 2:
            html_output += "<h4>Differences</h4>"

            text1 = reports_df[reports_df['llm_provider'] == providers[0]].iloc[0][section]
            text2 = reports_df[reports_df['llm_provider'] == providers[1]].iloc[0][section]

            differ = HtmlDiff(tabsize=4)
            diff_html = differ.make_table(text1.splitlines(), text2.splitlines(),
                                          providers[0], providers[1],
                                          context=True, numlines=3)

            html_output += f"<div style='font-size:0.8em; overflow-x:scroll;'>{diff_html}</div>"

    # Add similarity metrics
    html_output += "<h3>Similarity Analysis</h3>"

    similarity_data = {}
    for section in sections:
        text1 = reports_df[reports_df['llm_provider'] == providers[0]].iloc[0][section]
        text2 = reports_df[reports_df['llm_provider'] == providers[1]].iloc[0][section]

        similarity = SequenceMatcher(None, text1, text2).ratio()
        similarity_data[section] = similarity

    similarity_df = pd.DataFrame([similarity_data])

    # Convert dataframe to HTML table
    similarity_html = similarity_df.to_html(index=False)
    html_output += similarity_html

    # Add a bar chart
    plt.figure(figsize=(10, 4))
    plt.bar(similarity_data.keys(), similarity_data.values())
    plt.ylim(0, 1)
    plt.title('Text Similarity Between Providers')
    plt.ylabel('Similarity (0-1)')
    plt.savefig('similarity_chart.png')
    plt.close()

    html_output += "<img src='similarity_chart.png' style='max-width:600px;'>"

    return HTML(html_output)


# Example usage - run this cell to compare reports
reports = compare_reports_for_same_document()
display_report_comparison(reports)


# Verification of Different LLMs
#
# The comparison above should reveal whether the reports are truly generated by different
# LLMs. Look for:
#
# 1. **Different writing styles** - Each LLM has distinct patterns
# 2. **Different emphasis** - What facts each model focuses on
# 3. **Confidence scores** - Based on language pattern analysis
#
# If the reports are nearly identical, they may be coming from the same LLM despite
# different provider settings in your configuration.

def check_llm_configuration():
    """Check the current LLM configuration in the .env file"""
    provider = os.getenv("LLM_PROVIDER", "").lower()
    openai_key = os.getenv("OPENAI_API_KEY", "")
    google_key = os.getenv("GOOGLE_API_KEY", "")

    print(f"Current LLM provider: {provider}")
    print(f"OpenAI API key configured: {'Yes' if openai_key else 'No'}")
    print(f"Google API key configured: {'Yes' if google_key else 'No'}")

    # Check for potential issues
    issues = []
    if not provider:
        issues.append("LLM_PROVIDER is not set")
    if provider == "openai" and not openai_key:
        issues.append("Using OpenAI provider but API key is missing")
    if provider == "google" and not google_key:
        issues.append("Using Google provider but API key is missing")

    if issues:
        print("\nPotential issues:")
        for issue in issues:
            print(f"- {issue}")
    else:
        print("\nConfiguration looks valid.")


check_llm_configuration()